In [ ]:
import os
import ee
import geemap
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
# import datetime
import warnings
warnings.filterwarnings("ignore")
import sys
# print(sys.getrecursionlimit())
sys.setrecursionlimit(1000000)

In [ ]:
#GEE Authentication
# Either use the credentials approach using a json file or Authenticate
# service_account = 'serviceaccount.com'
# credentials = ee.ServiceAccountCredentials(service_account, 'location/to/json/file.json')
# ee.Initialize(credentials)

In [ ]:
ee.Initialize()

In [ ]:
ee.Authenticate()

In [ ]:
# Input Shapefile Address
file_path = r"Input\SHapefile\location.shp"
AOI = geemap.shp_to_ee(file_path)

In [ ]:
# This function clips all images to AOI
def clipper(image):
    return image.clip(aoi)

In [ ]:
# This part of the code tries to generate a list of dates that the collection will be created upon
# This approach is useful when the collection may be too big to be computed at once
def date_list_maker(st0,et0):
    from datetime import datetime,timedelta
    
    # Convert input strings to datetime objects
    start_date = datetime.strptime(st0, "%Y-%m-%d")
    end_date = datetime.strptime(et0, "%Y-%m-%d")
    
    # Initialize a list to store the result
    date_range = []
    
    # Define the interval in days (60 days in this case)
    interval = timedelta(days=60)
    
    # Start with the input start date and add intervals until it's less than or equal to the end date
    current_date = start_date
    while current_date <= end_date:
        next_date = current_date + interval
        if next_date <= end_date:
            date_range.append((current_date.strftime("%Y-%m-%d"), next_date.strftime("%Y-%m-%d")))
        else:
            date_range.append((current_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")))
        current_date = next_date
    
    return date_range

from datetime import date,timedelta
import datetime

# df0 = pd.DataFrame()
def sabkucch_2(st,et):
    
    # Define the collection
    dataset = ee.ImageCollection(collection) \
                    .filterDate(st, et) \
                    .filterBounds(aoi) \
                    .select(band) \
                    .map(clipper)
    
    # This function adds the mean, upper and lower percentile of each image into the metadata of the image
    def func_fsn(img):
        mn = img.reduceRegion(ee.Reducer.mean(),aoi).get(band)
        bp = ee.Image(img.reduceRegion(ee.Reducer.percentile([5]),aoi).get(band))
        up = ee.Image(img.reduceRegion(ee.Reducer.percentile([95]),aoi).get(band))
        return img.set({ 
            'mn': mn,
            'bp': bp,
            'up': up
        })
    
    dataset = dataset.map(func_fsn)
    
    # Filtering images that do not have mean, upper and lower percentile, i.e., images that do not have pixels in the AOI
    dataset = dataset.filterMetadata('mn', 'not_equals', None) \
                    .filterMetadata('bp', 'not_equals', None) \
                       .filterMetadata('up', 'not_equals', None)

    # Removes extreme values in the image
    def chipper(image):
        bp = ee.Image(image.reduceRegion(ee.Reducer.percentile([5]),aoi).get(band))
        up = ee.Image(image.reduceRegion(ee.Reducer.percentile([95]),aoi).get(band))
        a = image.select(band)
        mask = a.gte(ee.Number(bp)).And(a.lte(ee.Number(up))).And(a.gte(ee.Number(0)))
        return image.updateMask(mask)
    
    dataset = dataset.map(chipper)
    
    # dataset.size().getInfo()
    
    # dataset.getInfo()
    
    # def index_maker(col0):
    # Function to add a Moving Index Property to an image
    def addIndex(image, newIndex):
        return image.set('Moving_index', newIndex)
    
    # Iterate through the collection and add the property
    # Declaring Empty Collection to be filled after adding Moving Index Property
    X_collection = ee.ImageCollection([])
    newIndex = 0
    for i in range(dataset.size().getInfo()):
        image = ee.Image(dataset.toList(dataset.size()).get(i))
        modified_image = addIndex(image, newIndex)
        X_collection = X_collection.merge(ee.ImageCollection([modified_image]))
        newIndex += 1
    
    def process_list(list_data):
            def compute_daily_mean(image):
                date = image.date().format('YYYY-MM-dd')
                mean_CO = image.reduceRegion(ee.Reducer.mean(), AOI).get(band)
                # return ee.List({'date': date, 'mean_CO': mean_CO})
                if mean_CO is None:
                    return ee.Feature(None, {'Date': date, 'Mean': -9999})
                else:
                    return ee.Feature(None, {'Date': date, 'Mean': mean_CO})
            
        
            daily_means = list_data.map(compute_daily_mean)
            
            # daily_means
            
            df = pd.DataFrame(daily_means.getInfo()['features'])
            # Initialize empty lists to store extracted 'date' and 'mean_CO'
            dates = []
            mean_COs = []
            
            # Iterate through the 'properties' column and extract 'date' and 'mean_CO'
            for prop in df['properties']:
                if 'Mean' in prop:
                    mean_COs.append(prop['Mean'])
                    dates.append(prop['Date'])
            
            # Create a new DataFrame with 'date' and 'mean_CO' columns
            new_df = pd.DataFrame({'Date': dates, 'Mean': mean_COs})
            
            # Reset the index of the new DataFrame
            new_df.reset_index(drop=True, inplace=True)
            
            # Print the resulting DataFrame
            # print(new_df)
            return new_df
    
    df = process_list(X_collection)
    
    return df

In [ ]:
st0 = '2020-01-01'
et0 = '2022-01-01'
kryptonite_no = 52
yr = 2022
aoi = AOI
gas = 'CO'
factor = 1000
collection = 'COPERNICUS/S5P/OFFL/L3_CO'
band = 'CO_column_number_density'

In [ ]:
current_working_directory = os.getcwd()

In [ ]:
# yr,aoi,gas,collection,band = 2022,BDA,'SO2',"COPERNICUS/S5P/NRTI/L3_SO2",'SO2_column_number_density'
for i in range(int(st0[0:4]),int(et0[0:4])+1):
    print(datetime.datetime.now(),'Started',str(i)+'-01-01')
    date_range_list = date_list_maker(str(i)+'-01-01',str(i+1)+'-01-01')
    df0 = pd.DataFrame()
    for st_d, et_d in date_range_list:
        dfx = sabkucch_2(st_d,et_d)
        df0 = pd.concat([df0, dfx], ignore_index=True)
        print(datetime.datetime.now(),'Set')
    fn = str(i)
    try:
        merged_doc_directory = os.path.join(current_working_directory,'Result')
        os.makedirs(merged_doc_directory)
    except:
        merged_doc_directory = os.path.join(current_working_directory,'Result')

    df0['Mean'] = df0['Mean']*factor
    df0.to_csv(os.path.join(merged_doc_directory,f'{gas}_{fn}.csv'))